In [129]:
from app import Errors
import itertools
import pandas as pd
import os
from math import pi
from datetime import timedelta

In [151]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import show, output_notebook, figure
from bokeh.palettes import Category20c
from bokeh import palettes
from bokeh import layouts
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

In [28]:
raw_data = Errors().error_heatmap()

In [29]:
error_heatmap = raw_data
error_heatmap['location'] = error_heatmap.apply(lambda row: f"{os.path.basename(row['filename'])}:{row['line_number']}", axis=1)
error_heatmap = error_heatmap.drop(columns=['filename', 'line_number'])
error_heatmap

error_count  end_of_day         location
device        date                                                
PTL_RD_AT_003 2020-02-24            3  2020-02-25  Database.py:111
PTL_RD_AT_000 2020-02-24            1  2020-02-25  Database.py:104
              2020-02-24            6  2020-02-25  Database.py:111
              2020-02-24            5  2020-02-25  Database.py:113
              2020-02-24            1  2020-02-25  Database.py:117
...                               ...         ...              ...
PTL_RD_ES_008 2020-03-13            2  2020-03-14  Database.py:111
PTL_RD_ES_009 2020-03-13            1  2020-03-14  Database.py:111
PTL_RD_ES_012 2020-03-13            2  2020-03-14  Database.py:111
PTL_RD_ES_006 2020-03-13            1  2020-03-14  watchdog.py:282
              2020-03-13            1  2020-03-14  watchdog.py:301

[153 rows x 3 columns]

In [143]:
def color_palette(size):
    palette_generator = itertools.cycle(palettes.Category20[20] + palettes.Set3[12] + palettes.Category20b[20])
    c = [color for color, _ in zip(palette_generator, range(num_locations))]
    return c

In [144]:
locations = pd.DataFrame(error_heatmap.location.unique(), columns=['location'])
locations['colors'] = palette(len(locations.location))
locations

,location,colors
0,Database.py:111,#1f77b4
1,Database.py:104,#aec7e8
2,Database.py:113,#ff7f0e
3,Database.py:117,#ffbb78
4,base_events.py:1285,#2ca02c
5,logging_test.py:53,#98df8a
6,Serial.py:132,#d62728
7,watchdog.py:95,#ff9896
8,watchdog.py:129,#9467bd
9,Lighting.py:284,#c5b0d5


In [145]:
eh = error_heatmap.reset_index()
errors_by_day = eh.groupby(['device', 'date']).sum().rename(columns=dict(error_count='errors_by_day'))
eh = eh.join(errors_by_day, on=['device', 'date'])
eh['error_count_normalized'] = eh.error_count / eh.errors_by_day
eh = eh.merge(locations, on = ['location'])
eh = eh.set_index(['device', 'date', eh.index]).sort_index()
eh

error_count  end_of_day         location  \
device        date                                                       
PTL_LT_AT_001 2020-03-09 94            17  2020-03-10     Serial.py:44   
PTL_RD_AT_000 2020-02-24 1              6  2020-02-25  Database.py:111   
                         74             1  2020-02-25  Database.py:104   
                         75             5  2020-02-25  Database.py:113   
                         76             1  2020-02-25  Database.py:117   
...                                   ...         ...              ...   
PTL_RD_ES_012 2020-03-12 126            2  2020-03-13  watchdog.py:261   
                         146            2  2020-03-13  watchdog.py:301   
              2020-03-13 73             2  2020-03-14  Database.py:111   
                         151            5  2020-03-14    Serial.py:155   
                         152            5  2020-03-14     Serial.py:50   

                              errors_by_day  error_count_normalized   colors  
device        date                                                            
PTL_LT_AT_001 2020-03-09 94              17                1.000000  #7f7f7f  
PTL_RD_AT_000 2020-02-24 1               15                0.400000  #1f77b4  
                         74              15                0.066667  #aec7e8  
                         75              15                0.333333  #ff7f0e  
                         76              15                0.066667  #ffbb78  
...                                     ...                     ...      ...  
PTL_RD_ES_012 2020-03-12 126             75                0.026667  #dbdb8d  
                         146             75                0.026667  #9edae5  
              2020-03-13 73              12                0.166667  #1f77b4  
                         151             12                0.416667  #ffffb3  
                         152             12                0.416667  #bebada  

[153 rows x 6 columns]

In [164]:
dates = eh.index.get_level_values(1).unique().sort_values()
x_range = min(dates) - timedelta(hours=12), max(dates) + timedelta(hours=12)

In [165]:
figures = {}

for device in eh.index.get_level_values(0).unique():
    p = figure(plot_height=300, plot_width=500, title=f"Error heatmap for {device}", toolbar_location=None,
               tools="hover", tooltips="@location: @error_count Errors", x_axis_type='datetime', x_range=x_range)

    device_data = eh.loc[device]
    for date in device_data.index.get_level_values(0).unique():
        data_source = ColumnDataSource(device_data.loc[date])
        p.vbar(bottom=cumsum('error_count_normalized', include_zero=True),
               top=cumsum('error_count_normalized'),
               x=date, width=timedelta(days=1)/2, source=data_source, fill_color='colors')

    figures[device] = p

In [166]:
show(layouts.column(*list(figures.values())))

# Compute time intervals where the ptl was online

In [1]:
from app import db

In [2]:
from app import DeadManPackage

In [4]:
query = db.session.query(DeadManPackage.device, DeadManPackage.timestamp)

In [5]:
import pandas as pd
from datetime import timedelta

In [79]:
data = pd.DataFrame(query.all())
data['delay'] = data.groupby('device').timestamp.diff()
data = data.fillna(timedelta(minutes=2))
data = data[data.delay > timedelta(seconds=90)]
data = data.rename(columns=dict(timestamp='begin'))
data = data.sort_values(['device', 'begin'], ascending=False)
data['duration'] = data.groupby('device').begin.diff().abs()
data['end'] = data.begin + data.duration
data = data.sort_values(['device', 'begin'], ascending=True)
data = data.set_index(['device', data.index]).sort_index()
data

begin           delay  \
device                                                           
PTL_RD_AT_000 80    2020-03-12 12:25:35.196265        00:02:00   
              996   2020-03-12 15:31:02.469141 00:01:53.820413   
              10327 2020-03-13 01:02:02.046814 00:02:14.093244   
              21419 2020-03-13 12:45:57.403024 00:02:45.886590   
              33818 2020-03-14 01:02:01.193043 00:02:47.392036   
PTL_RD_AT_001 0     2020-03-12 11:01:27.834999        00:02:00   
              75    2020-03-12 12:18:17.408448 00:02:36.370863   
              78    2020-03-12 12:23:48.719316 00:03:30.691471   
              997   2020-03-12 15:31:09.530427 00:02:48.684818   
              10324 2020-03-13 01:02:01.416997 00:02:11.499660   
              21209 2020-03-13 12:29:58.724728 00:01:56.018869   
              21428 2020-03-13 12:47:24.710999 00:03:23.034242   
              21915 2020-03-13 13:23:00.691520 00:02:31.282048   
PTL_RD_AT_002 82    2020-03-12 12:25:51.745822        00:02:00   
              998   2020-03-12 15:31:18.475507 00:01:53.143676   
              10314 2020-03-13 01:01:58.912183 00:01:55.920917   
              21430 2020-03-13 12:47:32.925237 00:02:25.837635   
              33807 2020-03-14 01:01:58.998564 00:03:15.550845   
PTL_RD_AT_003 83    2020-03-12 12:26:01.645620        00:02:00   
              753   2020-03-12 14:40:38.256750 00:02:13.512404   
              999   2020-03-12 15:31:26.030596 00:02:40.092493   
              10319 2020-03-13 01:02:00.211606 00:01:53.823667   
              21431 2020-03-13 12:47:41.990757 00:02:38.781347   
              33805 2020-03-14 01:01:58.142665 00:03:03.081629   
PTL_RD_AT_004 84    2020-03-12 12:26:13.972698        00:02:00   
              1000  2020-03-12 15:31:33.832853 00:02:48.002947   
              2404  2020-03-12 17:17:37.243323 00:02:46.269391   
              3057  2020-03-12 18:07:50.214182 00:02:04.036573   
              3379  2020-03-12 18:33:41.256168 00:01:47.195769   
              10317 2020-03-13 01:01:59.502880 00:02:07.912763   
...                                        ...             ...   
PTL_RD_ES_007 21459 2020-03-13 12:50:35.206183 02:52:56.689309   
              32861 2020-03-14 00:04:16.107982 00:02:37.156716   
              33806 2020-03-14 01:01:58.071574 00:02:31.636519   
PTL_RD_ES_008 1013  2020-03-12 15:32:50.559817        00:02:00   
              10320 2020-03-13 01:02:00.510329 00:23:30.300913   
              21462 2020-03-13 12:50:45.787628 01:28:52.798819   
              32867 2020-03-14 00:04:24.811360 00:02:35.904264   
              33804 2020-03-14 01:01:57.436555 00:02:22.248495   
PTL_RD_ES_009 1015  2020-03-12 15:32:59.576876        00:02:00   
              10318 2020-03-13 01:01:59.403197 00:23:18.124496   
              21464 2020-03-13 12:50:53.770433 01:25:01.790674   
              32859 2020-03-14 00:04:13.433853 00:02:15.424934   
              33811 2020-03-14 01:01:59.369593 00:02:35.562758   
PTL_RD_ES_010 1018  2020-03-12 15:33:09.130129        00:02:00   
              10321 2020-03-13 01:02:00.948585 00:02:11.521201   
              21467 2020-03-13 12:51:00.294924 01:16:08.962694   
              33810 2020-03-14 01:01:59.368109 00:02:27.882493   
PTL_RD_ES_011 1021  2020-03-12 15:33:15.211378        00:02:00   
              10316 2020-03-13 01:01:59.124246 00:02:05.891761   
              21469 2020-03-13 12:51:10.272305 02:06:27.879794   
              32858 2020-03-14 00:04:11.227241 00:01:58.571545   
              33816 2020-03-14 01:02:00.457821 00:02:38.687015   
PTL_RD_ES_012 1024  2020-03-12 15:33:25.502119        00:02:00   
              2274  2020-03-12 17:06:41.236337 00:52:09.410872   
              3627  2020-03-12 18:50:54.718539 00:58:06.842775   
              10330 2020-03-13 01:02:01.934348 00:01:59.225369   
              21362 2020-03-13 12:41:43.172843 00:02:36.013582   
              21472 2020-03-13 12:51:17.443848 00:03:33.063750   
              3

In [75]:
data.end = data.end.abs()

In [76]:
data

begin           delay             end
device                                                                        
PTL_RD_AT_000 80    2020-03-12 12:25:35.196265        00:02:00 03:05:27.272876
              996   2020-03-12 15:31:02.469141 00:01:53.820413 09:30:59.577673
              10327 2020-03-13 01:02:02.046814 00:02:14.093244 11:43:55.356210
              21419 2020-03-13 12:45:57.403024 00:02:45.886590 12:16:03.790019
              33818 2020-03-14 01:02:01.193043 00:02:47.392036             NaT
PTL_RD_AT_001 0     2020-03-12 11:01:27.834999        00:02:00 01:16:49.573449
              75    2020-03-12 12:18:17.408448 00:02:36.370863 00:05:31.310868
              78    2020-03-12 12:23:48.719316 00:03:30.691471 03:07:20.811111
              997   2020-03-12 15:31:09.530427 00:02:48.684818 09:30:51.886570
              10324 2020-03-13 01:02:01.416997 00:02:11.499660 11:27:57.307731
              21209 2020-03-13 12:29:58.724728 00:01:56.018869 00:17:25.986271
              21428 2020-03-13 12:47:24.710999 00:03:23.034242 00:35:35.980521
              21915 2020-03-13 13:23:00.691520 00:02:31.282048             NaT
PTL_RD_AT_002 82    2020-03-12 12:25:51.745822        00:02:00 03:05:26.729685
              998   2020-03-12 15:31:18.475507 00:01:53.143676 09:30:40.436676
              10314 2020-03-13 01:01:58.912183 00:01:55.920917 11:45:34.013054
              21430 2020-03-13 12:47:32.925237 00:02:25.837635 12:14:26.073327
              33807 2020-03-14 01:01:58.998564 00:03:15.550845             NaT
PTL_RD_AT_003 83    2020-03-12 12:26:01.645620        00:02:00 02:14:36.611130
              753   2020-03-12 14:40:38.256750 00:02:13.512404 00:50:47.773846
              999   2020-03-12 15:31:26.030596 00:02:40.092493 09:30:34.181010
              10319 2020-03-13 01:02:00.211606 00:01:53.823667 11:45:41.779151
              21431 2020-03-13 12:47:41.990757 00:02:38.781347 12:14:16.151908
              33805 2020-03-14 01:01:58.142665 00:03:03.081629             NaT
PTL_RD_AT_004 84    2020-03-12 12:26:13.972698        00:02:00 03:05:19.860155
              1000  2020-03-12 15:31:33.832853 00:02:48.002947 01:46:03.410470
              2404  2020-03-12 17:17:37.243323 00:02:46.269391 00:50:12.970859
              3057  2020-03-12 18:07:50.214182 00:02:04.036573 00:25:51.041986
              3379  2020-03-12 18:33:41.256168 00:01:47.195769 06:28:18.246712
              10317 2020-03-13 01:01:59.502880 00:02:07.912763 11:45:47.936992
...                                        ...             ...             ...
PTL_RD_ES_007 21459 2020-03-13 12:50:35.206183 02:52:56.689309 11:13:40.901799
              32861 2020-03-14 00:04:16.107982 00:02:37.156716 00:57:41.963592
              33806 2020-03-14 01:01:58.071574 00:02:31.636519             NaT
PTL_RD_ES_008 1013  2020-03-12 15:32:50.559817        00:02:00 09:29:09.950512
              10320 2020-03-13 01:02:00.510329 00:23:30.300913 11:48:45.277299
              21462 2020-03-13 12:50:45.787628 01:28:52.798819 11:13:39.023732
              32867 2020-03-14 00:04:24.811360 00:02:35.904264 00:57:32.625195
              33804 2020-03-14 01:01:57.436555 00:02:22.248495             NaT
PTL_RD_ES_009 1015  2020-03-12 15:32:59.576876        00:02:00 09:28:59.826321
              10318 2020-03-13 01:01:59.403197 00:23:18.124496 11:48:54.367236
              21464 2020-03-13 12:50:53.770433 01:25:01.790674 11:13:19.663420
              32859 2020-03-14 00:04:13.433853 00:02:15.424934 00:57:45.935740
              33811 2020-03-14 01:01:59.369593 00:02:35.562758             NaT
PTL_RD_ES_010 1018  2020-03-12 15:33:09.130129        00:02:00 09:28:51.818456
              10321 2020-03-13 01:02:00.948585 00:02:11.521201 11:48:59.346339
              21467 2020-03-13 12:51:00.294924 01:16:08.962694 12:10:59.073185
              33810 2020-03-14 01:01:59.368109 00:02:27.882493             NaT
PTL_RD_ES_011 1021  2020-03-12 15:33:15.211378        00:02:00 09:28:43.912868
            

In [30]:
data.loc['PTL_RD_AT_004']

,timestamp,delay,data_loss
84,2020-03-12 12:26:13.972698,00:00:00,False
89,2020-03-12 12:27:14.025521,00:01:00.052823,False
94,2020-03-12 12:28:14.361095,00:01:00.335574,False
99,2020-03-12 12:29:14.527225,00:01:00.166130,False
104,2020-03-12 12:30:14.678421,00:01:00.151196,False
109,2020-03-12 12:31:14.854725,00:01:00.176304,False
114,2020-03-12 12:32:14.946775,00:01:00.092050,False
119,2020-03-12 12:33:15.088827,00:01:00.142052,False
124,2020-03-12 12:34:15.155804,00:01:00.066977,False
129,2020-03-12 12:35:15.265048,00:01:00.109244,False


timestamp           delay  data_loss
device                                                                   
PTL_RD_AT_001 75    2020-03-12 12:18:17.408448 00:02:36.370863       True
              78    2020-03-12 12:23:48.719316 00:03:30.691471       True
PTL_RD_AT_003 753   2020-03-12 14:40:38.256750 00:02:13.512404       True
PTL_RD_AT_000 996   2020-03-12 15:31:02.469141 00:01:53.820413       True
PTL_RD_AT_001 997   2020-03-12 15:31:09.530427 00:02:48.684818       True
PTL_RD_AT_002 998   2020-03-12 15:31:18.475507 00:01:53.143676       True
PTL_RD_AT_003 999   2020-03-12 15:31:26.030596 00:02:40.092493       True
PTL_RD_AT_004 1000  2020-03-12 15:31:33.832853 00:02:48.002947       True
PTL_RD_ES_007 1818  2020-03-12 16:32:37.125094 00:33:50.517917       True
PTL_RD_ES_001 2175  2020-03-12 16:59:07.984816 01:00:11.566188       True
PTL_RD_ES_012 2274  2020-03-12 17:06:41.236337 00:52:09.410872       True
PTL_RD_AT_004 2404  2020-03-12 17:17:37.243323 00:02:46.269391       True
              3057  2020-03-12 18:07:50.214182 00:02:04.036573       True
PTL_RD_ES_007 3086  2020-03-12 18:10:40.369815 00:06:46.578132       True
PTL_RD_AT_004 3379  2020-03-12 18:33:41.256168 00:01:47.195769       True
PTL_RD_ES_012 3627  2020-03-12 18:50:54.718539 00:58:06.842775       True
PTL_RD_ES_006 9173  2020-03-13 00:04:07.198444 00:37:04.811922       True
PTL_RD_AT_002 10314 2020-03-13 01:01:58.912183 00:01:55.920917       True
PTL_RD_ES_003 10315 2020-03-13 01:01:58.870974 00:02:01.756986       True
PTL_RD_ES_011 10316 2020-03-13 01:01:59.124246 00:02:05.891761       True
PTL_RD_AT_004 10317 2020-03-13 01:01:59.502880 00:02:07.912763       True
PTL_RD_ES_009 10318 2020-03-13 01:01:59.403197 00:23:18.124496       True
PTL_RD_AT_003 10319 2020-03-13 01:02:00.211606 00:01:53.823667       True
PTL_RD_ES_008 10320 2020-03-13 01:02:00.510329 00:23:30.300913       True
PTL_RD_ES_010 10321 2020-03-13 01:02:00.948585 00:02:11.521201       True
PTL_RD_ES_005 10322 2020-03-13 01:02:00.768576 00:01:53.295768       True
PTL_RD_ES_007 10323 2020-03-13 01:02:00.498871 00:02:03.614822       True
PTL_RD_AT_001 10324 2020-03-13 01:02:01.416997 00:02:11.499660       True
PTL_RD_ES_002 10325 2020-03-13 01:02:00.241515 00:02:11.245400       True
PTL_RD_ES_004 10326 2020-03-13 01:01:59.989549 00:01:59.504692       True
...                                        ...             ...        ...
PTL_RD_ES_011 21469 2020-03-13 12:51:10.272305 02:06:27.879794       True
PTL_RD_ES_012 21472 2020-03-13 12:51:17.443848 00:03:33.063750       True
PTL_RD_AT_001 21915 2020-03-13 13:23:00.691520 00:02:31.282048       True
PTL_RD_ES_002 21919 2020-03-13 13:20:59.234653 00:03:00.567673       True
PTL_RD_ES_006 32856 2020-03-14 00:04:07.099436 00:01:37.183724       True
PTL_RD_ES_004 32857 2020-03-14 00:04:10.847628 00:02:01.700327       True
PTL_RD_ES_011 32858 2020-03-14 00:04:11.227241 00:01:58.571545       True
PTL_RD_ES_009 32859 2020-03-14 00:04:13.433853 00:02:15.424934       True
PTL_RD_ES_007 32861 2020-03-14 00:04:16.107982 00:02:37.156716       True
PTL_RD_ES_012 32862 2020-03-14 00:04:17.519507 00:01:56.763576       True
PTL_RD_ES_005 32863 2020-03-14 00:04:18.753037 00:02:57.791831       True
PTL_RD_ES_003 32864 2020-03-14 00:04:21.713611 00:02:17.378894       True
PTL_RD_ES_001 32866 2020-03-14 00:04:22.779943 00:01:39.345153       True
PTL_RD_ES_008 32867 2020-03-14 00:04:24.811360 00:02:35.904264       True
              33804 2020-03-14 01:01:57.436555 00:02:22.248495       True
PTL_RD_AT_003 33805 2020-03-14 01:01:58.142665 00:03:03.081629       True
PTL_RD_ES_007 33806 2020-03-14 01:01:58.071574 00:02:31.636519       True
PTL_RD_AT_002 33807 2020-03-14 01:01:58.998564 00:03:15.550845       True
PTL_RD_ES_012 33808 2020-03-14 01:01:59.154695 00:02:30.757712       True
PTL_RD_ES_001 33809 2020-03-14 01:01:59.171495 00:02:25.015381       True
PTL_RD_ES_010 33810 2020-03-14 01:01:59.368109 00:02:27.882493       True
PTL_RD_ES_009 33811 2020-03-14 01:01:59.3